In [17]:
import openai
import nest_asyncio
nest_asyncio.apply()

In [18]:
openai.api_key = 'YOUR API KEY'

## Loading the data

In [19]:
from llama_index.core import SimpleDirectoryReader

# loading documents
documents = SimpleDirectoryReader(input_files=['cv.pdf']).load_data()

## Defining LLM and Embedding Model

In [20]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [21]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

## Defining Summary Index and Vector Index Over The Same Data

In [22]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

## Define Query Engines And Set MetaData

In [24]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,   
)
vector_query_engine = vector_index.as_query_engine()

In [26]:
from llama_index.core.tools import QueryEngineTool

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=("Useful for summarization questions related to CV "),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=("Useful for retrieving specific context from the provided CV.")
)


## Defining Router Query Engine

In [30]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [31]:
response = query_engine.query("What is the summary of the provided CV?")
print(str(response))

Selecting query engine 0: Useful for summarization questions related to CV.
The individual is a 3rd-year undergraduate student with a strong foundation in machine learning, data analysis, and programming. They are passionate about using data to solve real-world problems and are committed to continuous learning and professional growth in the fields of machine learning and data science. They have experience in developing basic machine learning models, data cleaning, preprocessing, and visualization. Proficient in Python and relevant libraries, they possess strong communication and teamwork skills, are self-motivated, and eager to learn. Additionally, they have engaged in various projects and training programs related to data analytics, machine learning, and deep learning, showcasing a dedication to expanding their knowledge and skills in these areas.


In [32]:
response = query_engine.query("List the courses/trainings the person has taken")
print(str(response))

Selecting query engine 1: This choice is more relevant as it focuses on retrieving specific context from the provided CV, which would include listing the courses/trainings the person has taken..
IBM Data Analytics With Excel And R Professional Certificate, Machine Learning Specialization, Machine Learning Scientist In Python, Deep Learning Specialization, NLP Specialization
